## Prétraitement des données

Ce notebook a pour objectif de mettre en œuvre les étapes de prétraitement nécessaires
avant la phase de modélisation. Il inclut la séparation des données en jeux
d’entraînement et de test, ainsi que la préparation des variables numériques
et catégorielles afin de garantir une modélisation reproductible et cohérente.


### 1- Packages et chargement des données 

In [36]:
from pathlib import Path
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE


In [9]:
DATA_RAW = Path("..") / "data" / "raw"
DATA_FILE = "eudirectlapse.csv"

df = pd.read_csv(DATA_RAW / DATA_FILE)
TARGET_COL = "lapse"

### 2- Séparation de la varaible cible et les variables explicatives 

In [10]:
X = df.drop(columns=[TARGET_COL]).copy()
y = df[TARGET_COL].copy()

print(X.shape, y.shape)
print(y.value_counts(normalize=True))

(23060, 18) (23060,)
lapse
0    0.871899
1    0.128101
Name: proportion, dtype: float64


### 3- Séparation jeu d'apprentissage et de test 

Les données sont séparées en un jeu d’entraînement (75 %) et un jeu de test (25 %) L'option stratidy = y permet de garantir que la proportion de résiliations soit la même dans le jeu de test et le jeu d'apprentissage. Autrement dit, la distribution de la variable lapse dans les deux bases est la même, environ 12,8 %.

L'option random_state = 42 permet de garantir la reproductibilité du code. 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    stratify=y,
    random_state=42
)

print("Train target rate:", y_train.mean())
print("Test target rate:", y_test.mean())

Train target rate: 0.1281295172015033
Test target rate: 0.1280138768430182


### 4-Identification des variables numériques et catégorielles 

In [15]:
num_cols = X_train.select_dtypes(include=["number", "bool"]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

print("# variables numériques :", len(num_cols))
print("# variables catégorielles :", len(cat_cols))


# variables numériques : 9
# variables catégorielles : 9


Le SMOTE ne s'applique que sur des variables numériques, puisque qu'il repose sur l'utilisation de la distances euclidiennes. Deux options sont alors envisageables : 

1 - SMOTE uniquement sur les variables numériques

2 - Encodage des variables catégoreilles puis SMOTE.

Dans la suite, nous nous concentrons dans un premier temps sur les variables numériques afin d’appliquer SMOTE dans un cadre respectant ses hypothèses théoriques.

In [16]:
#Permet d'isoler les variables numériques 
X_train_num = X_train[num_cols].copy()
X_test_num  = X_test[num_cols].copy()

print(X_train_num.shape, X_test_num.shape)


(17295, 9) (5765, 9)


In [ ]:
scaler = StandardScaler()

X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled  = scaler.transform(X_test_num)

Les variables numériques sont standardisées afin d’éviter que certaines variables dominent le calcul des distances utilisées par SMOTE.